In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Luminesce Syntax

Attributes
----------

"""

toggle_code("Toggle Docstring")

In [2]:
from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
import lusid as lu
import pandas as pd
import lusid_drive
import lusid_drive.rest
import lusid.rest
from lusid_drive.rest import ApiException
from pprint import pprint
import json

pd.set_option('display.max_colwidth', None)
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = ApiClientFactory(
    token = RefreshingToken(), 
    api_secrets_filename = secrets_path,
    app_name = "LusidJupyterNotebook"
)

# Confirm success by printing SDK version
api_status = pd.DataFrame(api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict())
display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.10321.0,0.5.3088,"{'relation': 'RequestLogs', 'href': 'http://fb..."


In [3]:
# here's some sample equities we've prepared earlier
uk_equity_instruments = pd.read_csv('_data/uk_equity_instruments.csv')
us_equity_instruments = pd.read_csv('_data/us_equity_instruments.csv')

In [4]:
# don't show exception if error is due to upsert
def exception_guard(e, code):
    return e.status and e.status != '400 Bad Request' and e.body and json.loads(e.body)["code"] == code

scope = 'FBNUniversity' # str | The scope of the portfolio.

In [5]:

# create a request to upsert a bunch of instruments.
def generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier):
    request_id = name + isin
    request ={"name":name,
                     "identifiers":{ "Figi":{"value":figi},
                                     "Isin":{"value":isin},
                                     other_identifier_name:{"value":other_identifier}
                                   },

                     "definition":{"instrumentType":"Equity", "domCcy":ccy}}
    return request_id, request

In [6]:
# Create an instance of the API class
api_instance = api_factory.build(lu.InstrumentsApi)

# We're going to load some american and UK equities into LUSID

other_identifier_name = 'Sedol'
ccy = 'GBP'
upsert_body_tuple = (generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier)
               for 
               index, ticker, name, _, isin, other_identifier, figi
               in uk_equity_instruments.itertuples())
uk_equity_body_dict = {request_id:request_body for request_id,request_body in upsert_body_tuple}
other_identifier_name = 'Cusip'
ccy = 'USD'
upsert_body_tuple = (generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier)
               for 
               index, ticker, name, _, isin, other_identifier, figi
               in us_equity_instruments.itertuples())
us_equity_body_dict = {request_id:request_body for request_id,request_body in upsert_body_tuple}

request_body_dict = us_equity_body_dict | uk_equity_body_dict

try:
    # UpsertInstruments: Upsert instruments
    api_response = api_instance.upsert_instruments(request_body_dict, scope=scope)
#     pprint(api_response)
except lusid.rest.ApiException as e:
    print("Exception when calling InstrumentsApi->upsert_instruments: %s\n" % e)

In [7]:
api_instance = api_factory.build(lu.TransactionPortfoliosApi)
code = 'Module-T2-3' # str | The code of the portfolio. Together with the scope this uniquely identifies the portfolio.
create_transaction_portfolio_request = {"displayName":"Training module T2.3", "baseCurrency":"GBP", "code":code} # UpdatePortfolioRequest | The updated portfolio definition.
try:
    # CreatePortfolio: Create portfolio
    api_response = api_instance.create_portfolio(scope, create_transaction_portfolio_request)
except lusid.rest.ApiException as e:
    if not exception_guard(e, 112):
        print(e)
        

In [8]:

# let's chuck a file containing a randomly selected subset of 25 instruments with their sectors into drive for some examples:

subset_file = open('_data/instrument_subset.csv').read()

drive_config = lusid_drive.Configuration(
    host = f'{api_instance.api_client.configuration.host[:-4]}/drive'
)
drive_config.access_token = RefreshingToken()

# Enter a context with an instance of the API client
with lusid_drive.ApiClient(drive_config) as api_client:
    # Create an instance of the API class
    api_instance = lusid_drive.FoldersApi(api_client)
    create_folder = {"path":"/","name":"finbourne university"} # CreateFolder | A CreateFolder object that defines the name and path of the new folder

    try:
        # [EARLY ACCESS] CreateFolder: Create a new folder in LUSID Drive
        api_response = api_instance.create_folder(create_folder)
        pprint(api_response)
    except ApiException as e:
        if not exception_guard(e, 664):
            print("Exception when calling FilesApi->create_file: %s\n" % e)
        
        
    api_instance = lusid_drive.FilesApi(api_client)
    x_lusid_drive_filename = 'instrument_subsetT2.3.csv' # str | File name.
    x_lusid_drive_path = '/finbourne university/' # str | File path.
    content_length = len(subset_file.encode('UTF-8'))
    body = subset_file # str | 

    try:
        # [EARLY ACCESS] CreateFile: Uploads a file to Lusid Drive. If using an SDK, consider using the UploadAsStreamAsync function for larger files instead.
        api_response = api_instance.create_file(x_lusid_drive_filename, x_lusid_drive_path, content_length, body)
    except lusid_drive.rest.ApiException as e:
        if not exception_guard(e, 671):
            print("Exception when calling FilesApi->create_file: %s\n" % e)

# Providers and Views


A provider is a component that enables you to write a Luminesce SQL query for a data source in situ, without first having to extract, transform or load data from that source.

We supply providers for numerous data sources, including for the investment data stored in LUSID itself and for files stored in Drive.

In this part of the course we will cover:

- Data providers
- Direct providers
- Write providers
- Provider parameters
- Using Luminesce views
- Writing Luminesce views

## Data providers

A data provider is designed to query a data source whose shape is known. It therefore returns a table of results with a fixed number of fields (columns).

Most of the providers we supply to query the investment management data stored in LUSID itself are data providers with a fixed number of fields, making it easier to query the data using standard SQL query syntax and luminesce extensions.

A data provider does not have to query a datasource, as long as it returns some data.For example, we supply providers that perform calculations on demand, such as Lusid.Portfolio.Valuation, where the data returned doesn't actually exist anywhere but rather is calculated each time the provider is queried.

## Direct providers 

A direct provider is designed to query a data source whose shape is not known, and thus cannot return a table of results with a fixed number of fields (columns).

A Luminesce query for a direct provider uses an arbitrary syntax, which may differ for each provider, and is defined by the writer of the provider. 

    @variable = use Some.Provider [wait]
    <arbitrary-syntax>
    enduse;
    select * from @variable


    
Some direct providers can also interpolate table and scalar parameters into their arbritrary syntax, enabling the use of parameters within the direct provider query using the WITH syntax:

    @@today = select strftime('%Y%m%d', 'now');
    @table_of_results = use Drive.Csv with @@today
    --file=/trade-files/eod-{@@today}.csv
    --noHeader
    --names=Equity,Units,Cost
    enduse;
    select * from @table_of_results


## Write providers

As mentioned earlier, data providers aren't limited to reading data from a datasource. Some providers' primary role is to write data to some target.

Using a write provider is straightforward. Query the Provider passing the data you'd like to write as parameters to fields specified by the provider, for example:

    @table_of_data = <select-statement>;
    select * from <Provider>.Writer where toWrite = @table_of_data;
    
By convention, many write providers accept a table of values as a parameter, and specify the expected columns in their fields.

## Examples of Lusid Providers

In this section we'll demonstrate some simple examples using the different types of provider.

### Data Providers


#### Lusid.Instrument.Equity Provider

Luminesce has a number of Lusid.Instrument providers that enable you to write a Luminesce query to retrieve instruments mastered in LUSID.

The generic Lusid.Instrument provider retrieves basic information about instruments of any asset class. Dedicated providers retrieve instruments of a particular asset class, for example Lusid.Instrument.Bond retrieves the full economic definition of bond instruments.

We'll use the Lusid.Instrument Provider to query for the equities we have stored in Lusid. Then we'll use the Lusid.Instrument.Equity Provider to look at the full economic definition of stored equity instruments.

In [9]:
%%luminesce
SELECT *
FROM Lusid.Instrument
LIMIT 100

,LusidInstrumentId,Isin,Sedol,Cusip,Ticker,ClientInternal,Figi,CompositeFigi,ShareClassFigi,Wertpapier,...,QuotePermId,EdiKey,Scope,DisplayName,LookThroughPortfolioScope,LookThroughPortfolioCode,Type,InferredAssetClass,InferredDomCcy,IsActive
0,LUID_00003DA2,US7223041028,NaN,722304102,NaN,NaN,BBG00LBLDDR2,NaN,NaN,NaN,...,NaN,NaN,fbn-uni-T02002,Pinduoduo Inc.,NaN,NaN,Equity,Equities,USD,True
1,CCY_MYR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,default,MYR,NaN,NaN,NaN,Unknown,ZZZ,True
2,LUID_00003DBG,US83088M1027,NaN,83088M102,NaN,NaN,BBG000GB4HW0,NaN,NaN,NaN,...,NaN,NaN,fbn-uni-T02002,"Skyworks Solutions, Inc.",NaN,NaN,Equity,Equities,USD,True
3,CCY_TTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,default,TTD,NaN,NaN,NaN,Unknown,ZZZ,True
4,CCY_CDF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,default,CDF,NaN,NaN,NaN,Unknown,ZZZ,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LUID_00003DE4,US8716071076,NaN,871607107,NaN,NaN,BBG000BSFRF3,NaN,NaN,NaN,...,NaN,NaN,FBNUniversity,"Synopsys, Inc.",NaN,NaN,Equity,Equities,USD,True
96,LUID_00003DAF,US0079031078,NaN,7903107,NaN,NaN,BBG000BBQCY0,NaN,NaN,NaN,...,NaN,NaN,fbn-uni-T02002,Advanced Micro Devices,NaN,NaN,Equity,Equities,USD,True
97,LUID_00003DLS,GB00B019KW72,B019KW7,NaN,NaN,NaN,BBG000BF0KW3,NaN,NaN,NaN,...,NaN,NaN,FBNUniversityModule-T2-2,Sainsbury's,NaN,NaN,Equity,Equities,GBP,True
98,CCY_ALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,default,ALL,NaN,NaN,NaN,Unknown,ZZZ,True


The Lusid.Instrument.Equity Provider has a couple of extra fields: DomCcy & Identifiers, and will only return Instruments which are equities.

In [10]:
%%luminesce
SELECT DisplayName, DomCcy
FROM Lusid.Instrument.Equity
LIMIT 100

,DisplayName,DomCcy
0,Pinduoduo Inc.,USD
1,"Skyworks Solutions, Inc.",USD
2,"PepsiCo, Inc.",USD
3,MercadoLibre,USD
4,Splunk,USD
...,...,...
95,Intermediate Capital Group,GBP
96,Coca-Cola HBC,GBP
97,"Facebook, Inc.",USD
98,Idexx Laboratories,USD


#### Lusid.Portfolio Provider

The Lusid.Portfolio provider enables you to write a Luminesce SQL query that retrieves information about one or more portfolios.

In [11]:
%%luminesce
SELECT *
FROM Lusid.Portfolio
LIMIT 100

,PortfolioScope,PortfolioCode,PortfolioType,BaseCurrency,DisplayName,Description,OriginPortfolioScope,OriginPortfolioCode,ParentPortfolioScope,ParentPortfolioCode,ChildPortfolioIds,CorporateActionSourceScope,CorporateActionSourceId,AccountingMethod,SubHoldingKeys,Created,InstrumentScopes
0,FBNUniversity,Module-6-4,Transaction,GBP,Training module 6.4 - Track commission as sepa...,NaN,FBNUniversity,Module-6-4,NaN,NaN,NaN,NaN,NaN,Default,Transaction/FBNUniversity/TrackCommission,2022-01-01,FBNUniversityModule-6-4
1,FBNUniversity,Module-6-1,Transaction,GBP,Training module 6.1 - Different transaction co...,NaN,FBNUniversity,Module-6-1,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-6-1
2,FBNUniversity,Module-6-3,Transaction,GBP,Training module 6.3 - Reduce cash balance by c...,NaN,FBNUniversity,Module-6-3,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-6-3
3,FBNUniversity,Module-3-1,Transaction,GBP,Training module 3.1,NaN,FBNUniversity,Module-3-1,NaN,NaN,NaN,NaN,NaN,Default,Transaction/FBNUniversity/Module-3-1,2022-01-01,FBNUniversityModule-3-1
4,FBNUniversity,Module-T2-2,Transaction,GBP,Training module T2.2,A portfolio to hold transactions,FBNUniversity,Module-T2-2,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-11-03 20:14:15.334,FBNUniversityModule-T2-2
5,FBNUniversity,Module-6-2,Transaction,GBP,"Training module 6.2 - Same transaction code, d...",NaN,FBNUniversity,Module-6-2,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-6-2
6,FBNUniversity,Module-1,Transaction,GBP,Module 1,NaN,FBNUniversity,Module-1,NaN,NaN,NaN,NaN,NaN,Default,NaN,2021-12-01,FBNUniversityModule-1
7,FBNUniversity,Module-T2-3,Transaction,GBP,Training module T2.3,NaN,FBNUniversity,Module-T2-3,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-11-21 15:59:45.669,NaN
8,FBNUniversity,Module-3-2,Transaction,GBP,Training module 3.2,NaN,FBNUniversity,Module-3-2,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-3-2
9,FBNUniversity,Module-4-1,Transaction,GBP,Training module 4.1,NaN,FBNUniversity,Module-4-1,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-4-1


Here we've returned all metadata about our Portfolios in Lusid.

#### Lusid.Tools Provider

The Lusid.Tools Provider does not query any data. It manipulates the input parameters to return a table. 

The Tools.JsonExpand provider enables you to write a Luminesce query that parses a JSON document into a table of constituent data objects, much like the SQLite json.tree function. We'll use this to demonstrate a data provider that does not query for any external data, instead manipulating the provided parameters.

In [12]:
%%luminesce
@@jsn = SELECT '{
    "characters":[
        {"name":"marge",
         "age":36},
        {"name":"homer",
         "age":40},
        {"name":"bart",
         "age":9},
        {"name":"lisa",
         "age":12}
    ]
}';
SELECT * FROM Tools.JsonExpand WHERE JsonString = @@jsn


,key,value,type,atom,id,parent,fullkey,path
0,NaN,"{""characters"":[{""name"":""marge"",""age"":36},{""nam...",object,NaN,0,NaN,$,$
1,characters,"[{""name"":""marge"",""age"":36},{""name"":""homer"",""ag...",array,NaN,2,0.0,$.characters,$
2,0,"{""name"":""marge"",""age"":36}",object,NaN,3,2.0,$.characters[0],$.characters
3,name,marge,text,marge,5,3.0,$.characters[0].name,$.characters[0]
4,age,36,integer,36,7,3.0,$.characters[0].age,$.characters[0]
5,1,"{""name"":""homer"",""age"":40}",object,NaN,8,2.0,$.characters[1],$.characters
6,name,homer,text,homer,10,8.0,$.characters[1].name,$.characters[1]
7,age,40,integer,40,12,8.0,$.characters[1].age,$.characters[1]
8,2,"{""name"":""bart"",""age"":9}",object,NaN,13,2.0,$.characters[2],$.characters
9,name,bart,text,bart,15,13.0,$.characters[2].name,$.characters[2]


In [13]:
%%luminesce
@@jsn = SELECT '{
    "characters":[
        {"name":"marge",
         "age":36},
        {"name":"homer",
         "age":40},
        {"name":"bart",
         "age":9},
        {"name":"lisa",
         "age":12}
    ]
}';
@@selection = select "$.characters[0]";
select * from Tools.JsonExpand where JsonString = @@jsn and RootPath = @@selection

,key,value,type,atom,id,parent,fullkey,path
0,NaN,"{""name"":""marge"",""age"":36}",object,NaN,3,NaN,$.characters[0],$.characters
1,name,marge,text,marge,5,3.0,$.characters[0].name,$.characters[0]
2,age,36,integer,36,7,3.0,$.characters[0].age,$.characters[0]


### Direct Providers

#### Drive.Csv Provider

The Drive.Csv provider enables you to write a Luminesce query that extracts data from one or more CSV or similar pipe-delimited or row-based text files stored in Drive.
We can pass a number of options to the Drive CSV provider, using the syntax `--<option>[=<value>]`

In [16]:
%%luminesce
@x = use Drive.Csv
--file=/finbourne university/instrument_subsetT2.3.csv
--noHeader
--names=Name,Sector,Identifier
enduse;
select * from @x

,Name,Sector,Identifier
0,Schroders,Financial Services,BBG000BF0TF3
1,DS Smith,General Industrials,BBG000BF1LF9
2,Flutter Entertainment,Travel & Leisure,BBG000DWL6M3
3,Scottish Mortgage Investment Trust,Equity Investment Instruments,BBG000BFZMY9
4,Taylor Wimpey,Household Goods & Home Construction,BBG000BF4KL1
5,BP,Oil & Gas Producers,BBG000C05BD1
6,Glencore,Mining,BBG001MM1KV4
7,Segro,Real Estate Investment Trusts,BBG000BF1HY7
8,Prudential plc,Life Insurance,BBG000BDY322
9,Pershing Square Holdings,Financial Services,BBG00747WHS1


Here we've read all data from the `/finbourne university/instrument_subsetT2.3.csv` file in drive. The file has no headers, so we've specified the no headers option, and provided a set of column names to the Provider.

### Write Providers

#### Lusid.Instrument.Equity.Writer

Many of our Lusid write providers accept a TableParameter used to write multiple resources to some part of Lusid.


Now we'll use a Write Provider to create some equities by passing a table of equities as a parameter.

First let's take a look at the table parameter used by the Lusid.Instruments.Equities provider, and what the table it expects looks like.

In [22]:
%%luminesce 
select FieldName, TableParamColumns from Sys.Field where TableName = 'Lusid.Instrument.Equity.Writer' AND DataType = 'Table';

,FieldName,TableParamColumns
0,ToWrite,LusidInstrumentId (Text)\nIsin (Text)\nSedol (Text)\nCusip (Text)\nTicker (Text)\nClientInternal (Text)\nFigi (Text)\nCompositeFigi (Text)\nShareClassFigi (Text)\nWertpapier (Text)\nRIC (Text)\nQuotePermId (Text)\nEdiKey (Text)\nScope (Text)\nDisplayName (Text)\nLookThroughPortfolioScope (Text)\nLookThroughPortfolioCode (Text)\nIdentifiers (Text)\nDomCcy (Text)


We can see there's a ToWrite column which accepts a table of equities. By convention, many Lusid write providers use toWrite as a table parameter, returning the written values in the table, and some Error information.

We recommend examining the results of every write query using one or more of the WriteError, WriteErrorCode and WriteErrorDetail fields.

For each record in the table of data to write, Lusid.Portfolio.Writer returns an error code. If the operation is successful, the error code is 0. If unsuccessful, a positive error code and explanation help you discover why LUSID considers the operation invalid.  

In [10]:
%%luminesce
select * from Sys.Field where TableName = 'Lusid.Instrument.Equity.Writer';

,TableName,FieldName,Source,DataType,FieldType,IsPrimaryKey,IsMain,ParamDefaultValue,TableParamColumns,Description,DisplayName,ConditionUsage,SampleValues,AllowedValues
0,Lusid.Instrument.Equity.Writer,ClientInternal,service-main,Text,Column,False,False,NaN,NaN,The instrument identifier ClientInternal,Client Internal,None,NaN,NaN
1,Lusid.Instrument.Equity.Writer,CompositeFigi,service-main,Text,Column,False,False,NaN,NaN,The instrument identifier CompositeFigi,Composite Figi,None,NaN,NaN
2,Lusid.Instrument.Equity.Writer,Cusip,service-main,Text,Column,False,False,NaN,NaN,The instrument identifier Cusip,Cusip,None,NaN,NaN
3,Lusid.Instrument.Equity.Writer,DeletePropertiesWhereNull,service-main,Boolean,Parameter,NaN,NaN,False,NaN,"If true, if there isn't a value specified for ...",Delete Properties Where Null,None,NaN,NaN
4,Lusid.Instrument.Equity.Writer,DisplayName,service-main,Text,Column,False,True,NaN,NaN,The instrument's display name.,Display Name,None,NaN,NaN
5,Lusid.Instrument.Equity.Writer,DomCcy,service-main,Text,Column,False,False,NaN,NaN,NaN,Dom Ccy,None,NaN,NaN
6,Lusid.Instrument.Equity.Writer,EdiKey,service-main,Text,Column,False,False,NaN,NaN,The instrument identifier EdiKey,Edi Key,None,NaN,NaN
7,Lusid.Instrument.Equity.Writer,EffectiveFrom,service-main,DateTime,Parameter,NaN,NaN,0001-01-01T00:00:00Z,NaN,Effective time to upsert at. The provider will...,Effective From,None,NaN,NaN
8,Lusid.Instrument.Equity.Writer,Figi,service-main,Text,Column,False,False,NaN,NaN,The instrument identifier Figi,Figi,None,NaN,NaN
9,Lusid.Instrument.Equity.Writer,Identifiers,service-main,Text,Column,False,False,NaN,NaN,NaN,Identifiers,None,NaN,NaN


We'll upsert some equities. Looking at the [knowledge-base](https://support.lusid.com/knowledgebase/article/KA-01685/en-us), we can see that the Equity.Writer Provider must have a DisplayName, at least one unique-identifier, and a DomCcy.

In [7]:
%%luminesce
@inserted_values = SELECT 
                    'T2.3-Fake-Equity-1' as DisplayName,
                    '0000000-000001' as Figi, 
                    'GBP' as DomCcy
                   UNION
                   SELECT
                   'T2.3 Fake Equity-2' as DisplayName,
                   '0000000-000002' as Figi,
                   'CHF' as DomCcy;

SELECT *
FROM @inserted_values;

,DisplayName,Figi,DomCcy
0,T2.3 Fake Equity-2,0000000-000002,CHF
1,T2.3-Fake-Equity-1,0000000-000001,GBP


In [9]:
%%luminesce
@inserted_values = SELECT 
                    'T2.3-Fake-Equity-1' as DisplayName,
                    '0000000-000001' as Figi, 
                    'GBP' as DomCcy
                   UNION
                   SELECT
                   'T2.3 Fake Equity-2' as DisplayName,
                   '0000000-000002' as Figi,
                   'CHF' as DomCcy;

SELECT * FROM Lusid.Instrument.Equity.Writer WHERE toWrite = @inserted_values;

,WriteErrorCode,WriteError,WriteErrorDetail,LusidInstrumentId,Isin,Sedol,Cusip,Ticker,ClientInternal,Figi,...,Scope,DisplayName,LookThroughPortfolioScope,LookThroughPortfolioCode,Type,InferredAssetClass,InferredDomCcy,IsActive,Identifiers,DomCcy
0,0,NaN,NaN,LUID_00003DON,NaN,NaN,NaN,NaN,NaN,0000000-000002,...,default,T2.3 Fake Equity-2,NaN,NaN,Equity,Equities,CHF,True,NaN,CHF
1,0,NaN,NaN,LUID_00003DOM,NaN,NaN,NaN,NaN,NaN,0000000-000001,...,default,T2.3-Fake-Equity-1,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP


As you can see, a table is returned with the values we've written, some values auto-created by Lusid, and error information (if there was an error).

#### Drive.SaveAs

The Drive.SaveAs Provider is a Direct Provider that allows you to write tables of data to different types of file. As its a direct provider that has its own arbritrary syntax used to tell the Provider what data to write, to which file type, and how to write the data.

In [11]:
%%luminesce
@inserted_file = SELECT
                'Marge' as CharacterName,
                '35' as Age, 
                'Married' as Status;
SELECT * FROM @inserted_file;

,CharacterName,Age,Status
0,Marge,35,Married


In [5]:
%%luminesce
@inserted_file = SELECT
                'Marge' as CharacterName,
                '35' as Age, 
                'Married' as Status;
                 
@x = use Drive.SaveAs with @inserted_file
--path=/finbourne university
--fileNames=simpsons
--type=Csv
enduse;
select * from @x

,VariableName,FileName,RowCount,Skipped
0,@inserted_file,/finbourne university/simpsons,1,False


We can see from the table that a simpsons.csv file has been written to drive, containing one row.

## Luminesce views

You can use the Sys.Admin.SetupView direct provider supplied by FINBOURNE to create a ‘view’ that can access any number of other Luminesce providers (or existing views).

The goal is to enable end users to write simplified or more performant queries using this view instead of the underlying providers directly, obscuring complexity, increasing productivity and promoting adoption.

Provider creation syntax

    @x = use Sys.Admin.SetupView [with @@scalar_var1 [, @@scalar_var2...]]
    <options-section>
    ----
    <sql-section>
    enduse;
    -- The following statement is optional but useful while creating the view to test it returns meaningful data
    select * from @x

In [9]:
%%luminesce
@x = use Sys.Admin.SetupView [with @@scalar_var1 [, @@scalar_var2...]]
--provider=Views.FBNUNIVIEW
----
SELECT #SELECT {
  { EquityName: i.DisplayName },
}
FROM Lusid.Instrument.Equity i
Limit 100
enduse;

,rowid_noop_alias


Here we've created a view called Views.FBNUNIVIEW which selects 100 DisplayNames from Lusid.Instrument.Equity and maps the DisplayName column to EquityName.

The #SELECT is special syntax used to optimize queries, described in the [setupview article](https://support.lusid.com/knowledgebase/article/KA-01767/en-us) in the knowledgebase.

We can now use this view to query data:

In [11]:
%%luminesce
SELECT * FROM Views.FBNUNIVIEW Limit 10;

,EquityName
0,Pinduoduo Inc.
1,Advanced Micro Devices
2,"Skyworks Solutions, Inc."
3,"PepsiCo, Inc."
4,Unilever
5,DexCom
6,Match Group
7,Splunk
8,Schroders
9,Antofagasta


We've selected 10 of the 100 Equities that would've been returned from our view.